# MOSTLY AI vs. SDV Comparison - Multi Table Scenario

This notebook provides a comprehensive comparison between two leading synthetic data generation platforms:
- **SDV (Synthetic Data Vault)** - Business Source License
- **MOSTLY AI Synthetic Data SDK** - Apache 2.0 License - Open Source
- **MOSTLY AI Synthetic Mock Data** - Apache 2.0 License - Open Source

## Dataset Overview

We'll be working with a realistic financial dataset consisting of two related tables:

1. **Customers Table** (`Transformed_Customer_Data.csv`): Contains comprehensive customer profiles including:
   - Personal information (name, age, gender, contact details)
   - Financial data (income, work details, credit card information)
   - Geographic data (address, coordinates)
   - Demographic information (education, marital status, race)

2. **Transfers Table** (`transfer_history_iter2.csv`): Contains money transfer transactions between customers:
   - Transaction metadata (ID, timestamp, amount)
   - Sender and receiver information (linked to customers via foreign keys)
   - Transaction notes

## Comparison Methodology

1. **Data Preparation**: Load, inspect, and preprocess the multi-table dataset
2. **Data Splitting**: Create train/test splits while maintaining referential integrity
3. **Model Training**: Train both SDV and MOSTLY AI generators on the training data
4. **Synthetic Data Generation**: Generate synthetic datasets using both platforms
5. **Performance Analysis**: Compare training time, generation speed, and data quality

## Key Challenges in Multi-Table Synthesis

- **Referential Integrity**: Maintaining foreign key relationships between tables
- **Sequential Dependencies**: Preserving temporal patterns in transaction data
- **Complex Relationships**: Handling multiple foreign keys (issuer_id and receiver_id both reference customers)
- **Data Quality**: Ensuring synthetic data maintains statistical properties and business logic


In [ ]:
# Install SDK in CLIENT mode
!uv pip install -U mostlyai
# Or install in LOCAL mode
!uv pip install -U 'mostlyai[local]'  
# Note: Restart kernel session after installation!

!uv pip install scikit-learn seaborn lightgbm sdv

## 1. Data Loading and Initial Exploration

First, let's load our multi-table dataset and examine its structure to understand:
- Table schemas and data types
- Data quality and completeness
- Relationships between tables
- Business logic and constraints


In [1]:
import time

import pandas as pd

In [ ]:
# Load the multi-table financial dataset
print("📂 Loading financial dataset...")
df_customers = pd.read_csv("./data/Transformed_Customer_Data.csv")
df_transfers = pd.read_csv("./data/transfer_history_iter2.csv")


def inspect_df(df, name):
    """
    Comprehensive data inspection function to understand:
    - Dataset dimensions and structure
    - Column names and data types
    - Sample data for manual review
    """
    print(f"--- {name} ---")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print("Columns:", df.columns.tolist())
    print("Dtypes:", df.dtypes)
    print("First 3 rows:")
    print(df.head(3))
    print("---\n")


# Inspect both tables to understand the data structure
print("🔍 Analyzing customer data structure...")
inspect_df(df_customers, "Transformed_Customer_Data")

print("🔍 Analyzing transfer data structure...")
inspect_df(df_transfers, "transfer_history_iter2")

## 2. Data Preprocessing: Establishing Foreign Key Relationships

The transfer data uses usernames as identifiers, but we need to establish proper foreign key relationships using customer IDs. This step is crucial for:
- Maintaining referential integrity in synthetic data
- Enabling proper multi-table synthesis
- Ensuring realistic transaction patterns


In [ ]:
print("🔗 Creating foreign key relationships...")

# Step 1: Create a lookup dictionary mapping username to customer_id
# This enables us to convert string-based identifiers to proper foreign keys
username_to_id = df_customers.set_index("username")["customer_id"].to_dict()
print(f"✅ Created mapping for {len(username_to_id):,} customers")

# Step 2: Transform transfer data to use customer_id foreign keys
# Replace username strings with integer customer IDs for both sender and receiver
print("🔄 Converting usernames to customer IDs in transfer data...")

df_transfers["issuer_id"] = df_transfers["issuer_id"].map(username_to_id)
df_transfers["receiver_id"] = df_transfers["receiver_id"].map(username_to_id)

# Data quality check: Verify all transfers have valid customer references
missing_issuer = df_transfers["issuer_id"].isna().sum()
missing_receiver = df_transfers["receiver_id"].isna().sum()

print("📊 Data Quality Report:")
print(f"   - Missing issuer mappings: {missing_issuer:,}")
print(f"   - Missing receiver mappings: {missing_receiver:,}")
print(f"   - Total transfers: {len(df_transfers):,}")

if missing_issuer == 0 and missing_receiver == 0:
    print("✅ Perfect referential integrity - all transfers linked to customers!")
else:
    print("⚠️ Some transfers reference non-existent customers")

# Display sample of transformed data
print("\n📋 Sample of transformed transfer data:")
missing_issuer, missing_receiver, df_transfers.head(3)

## 3. Strategic Data Splitting for Multi-Table Scenarios

When dealing with related tables, data splitting becomes more complex than simple random sampling. We need to:

**Key Considerations:**
- **Referential Integrity**: Ensure foreign key relationships remain valid in both splits
- **Business Logic**: Transactions can only exist between customers in the same split
- **Data Leakage Prevention**: Avoid information bleeding between train/test sets

**Our Approach:**
1. Split customers first (80/20 train/test)
2. Assign transfers based on participant customers
3. Transfers go to training set only if BOTH sender and receiver are in training set
4. All other transfers go to test set


In [ ]:
from sklearn.model_selection import train_test_split

print("✂️ Performing strategic multi-table data splitting...")

# Step 1: Split customers using 80/20 ratio
# Use random_state for reproducible results
customers_train, customers_test = train_test_split(df_customers, test_size=0.2, random_state=42, stratify=None)

print("👥 Customer split:")
print(f"   - Training set: {len(customers_train):,} customers ({len(customers_train) / len(df_customers) * 100:.1f}%)")
print(f"   - Test set: {len(customers_test):,} customers ({len(customers_test) / len(df_customers) * 100:.1f}%)")

# Step 2: Create customer ID sets for efficient lookup
train_ids = set(customers_train["customer_id"])
test_ids = set(customers_test["customer_id"])

# Step 3: Split transfers based on customer participation
# Training transfers: Both sender AND receiver must be in training customer set
# This ensures no data leakage and maintains business logic
transfers_train = df_transfers[
    df_transfers["issuer_id"].isin(train_ids) & df_transfers["receiver_id"].isin(train_ids)
].copy()

# Test transfers: All remaining transfers (at least one participant in test set)
transfers_test = df_transfers[
    ~(df_transfers["issuer_id"].isin(train_ids) & df_transfers["receiver_id"].isin(train_ids))
].copy()

print("💸 Transfer split:")
print(f"   - Training transfers: {len(transfers_train):,} ({len(transfers_train) / len(df_transfers) * 100:.1f}%)")
print(f"   - Test transfers: {len(transfers_test):,} ({len(transfers_test) / len(df_transfers) * 100:.1f}%)")

# Saving the splits
customers_train_output_file = "./data/customers_train.parquet"
customers_test_output_file = "./data/customers_test.parquet"
transfers_train_output_file = "./data/transfers_train.parquet"
transfers_test_output_file = "./data/transfers_test.parquet"
customers_train.to_parquet(customers_train_output_file, index=False)
customers_test.to_parquet(customers_test_output_file, index=False)
transfers_train.to_parquet(transfers_train_output_file, index=False)
transfers_test.to_parquet(transfers_test_output_file, index=False)

# Validate the split maintains referential integrity
train_senders_valid = transfers_train["issuer_id"].isin(train_ids).all()
train_receivers_valid = transfers_train["receiver_id"].isin(train_ids).all()

print("✅ Referential integrity check:")
print(f"   - All training senders in training customers: {train_senders_valid}")
print(f"   - All training receivers in training customers: {train_receivers_valid}")

if train_senders_valid and train_receivers_valid:
    print("🎯 Perfect data split - ready for training!")
else:
    print("⚠️ Data integrity issue detected!")

## 4. SDV (Synthetic Data Vault) Implementation

**About SDV:**
- Business Source License Python library for synthetic data generation
- Supports single-table and multi-table scenarios
- Uses statistical modeling and machine learning approaches
- Provides HMASynthesizer for hierarchical multi-table synthesis

**Key Features:**
- **Metadata Detection**: Automatically infers data types and relationships
- **Relationship Modeling**: Handles parent-child table relationships
- **Privacy Protection**: Generates synthetic data that preserves statistical properties while protecting individual privacy
- **Extensible**: Multiple synthesizer options (GaussianCopula, CTGAN, CopulaGAN, etc.)

**Limitations:**
- Current version only supports one parent per child table
- Complex multi-parent relationships require modeling simplification
- Performance scales with data complexity


### 4.1 SDV Metadata Configuration

The metadata configuration is crucial for SDV to understand:
- **Data Types**: Numerical, categorical, datetime, PII fields
- **Table Relationships**: Primary keys, foreign keys, and hierarchical structures
- **Constraints**: Business rules and data validation requirements

**Challenge**: Our transfers table has TWO foreign keys (issuer_id and receiver_id) both referencing the customers table. SDV v1.x only supports one parent per child, so we'll model one relationship explicitly.


In [ ]:
from sdv.metadata import Metadata
from sdv.multi_table import HMASynthesizer

print("🏗️ Building SDV metadata configuration...")

# Step 1: Auto-detect metadata from training dataframes
# SDV analyzes the data to infer column types, constraints, and potential relationships
metadata = Metadata.detect_from_dataframes(
    data={"customers": customers_train, "transfers": transfers_train}, infer_keys="primary_and_foreign"
)

print("✅ Base metadata auto-detected")

# Step 2: Define explicit table relationship
# Important limitation: SDV v1.x only supports one parent per child table
# We choose receiver_id as the primary relationship, issuer_id will be treated as a regular column
print("🔗 Configuring table relationships...")
print("   📌 Note: Due to SDV limitations, only modeling receiver_id → customer_id relationship")
print("   📌 issuer_id will be synthesized as a regular integer column")

metadata.add_relationship(
    parent_table_name="customers",
    child_table_name="transfers",
    parent_primary_key="customer_id",
    child_foreign_key="issuer_id",
)

print("✅ Relationship configured: customers → transfers (via issuer_id)")

# Display the complete metadata structure
print("\n📋 Complete SDV Metadata Configuration:")
metadata

### 4.2 SDV Model Training

**HMASynthesizer Overview:**
- **Hierarchical Modeling**: Learns parent-child relationships
- **Statistical Approach**: Uses copulas and Gaussian distributions
- **Multi-step Process**: 
  1. Preprocesses tables and infers constraints
  2. Learns relationships between parent and child tables
  3. Models individual table distributions
  
**Training Phases:**
- **Preprocess Tables**: Data cleaning and type inference
- **Learning Relationships**: Analyzing foreign key dependencies  
- **Modeling Tables**: Learning statistical distributions for synthesis


In [ ]:
print("🚀 Starting SDV training process...")
print("This will involve multiple phases - preprocessing, relationship learning, and table modeling")

start_time = time.time()

# Initialize the HMASynthesizer with our configured metadata
print("🔧 Initializing HMASynthesizer...")
synthesizer = HMASynthesizer(metadata)

# Fit the synthesizer on training data
# This process will:
# 1. Preprocess both tables (clean data, infer constraints)
# 2. Learn the customers → transfers relationship pattern
# 3. Model the statistical distributions of each table
print("📊 Training synthesizer on multi-table data...")
synthesizer.fit({"customers": customers_train, "transfers": transfers_train})

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print("✅ SDV training completed successfully!")
print(f"⏱️ Total training time: {elapsed_minutes:.2f} minutes")
print(f"📈 Training data: {len(customers_train):,} customers, {len(transfers_train):,} transfers")

### 4.3 SDV Synthetic Data Generation

**Generation Process:**
- **Scale Parameter**: Controls the number of synthetic records (1.0 = same size as training data)
- **Hierarchical Generation**: First generates parent records (customers), then child records (transfers)
- **Relationship Preservation**: Ensures all synthetic transfers reference valid synthetic customers
- **Statistical Sampling**: Uses learned distributions to create realistic synthetic data


In [ ]:
print("🎲 Starting SDV synthetic data generation...")
print("Generating synthetic data using learned statistical distributions...")

start_time = time.time()

# Generate synthetic data with 25% more records than training data
# Scale parameter: 1.0 = same size, 1.25 = 25% larger, 0.5 = half size
print("⚙️ Generating 1.25x the training data size...")
sdv_synthetic_data = synthesizer.sample(scale=1.25)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

# Calculate generation statistics
total_synthetic_records = len(sdv_synthetic_data["customers"]) + len(sdv_synthetic_data["transfers"])
generation_rate = total_synthetic_records / (end_time - start_time)

print("✅ SDV generation completed successfully!")
print(f"⏱️  Generation time: {elapsed_minutes:.2f} minutes")
print(f"🚀 Generation rate: {generation_rate:,.0f} records/second")
print(f"📊 Generated {len(sdv_synthetic_data['customers']):,} synthetic customers")
print(f"📊 Generated {len(sdv_synthetic_data['transfers']):,} synthetic transfers")

# Data quality verification
synthetic_customers_count = len(sdv_synthetic_data["customers"])
synthetic_transfers_count = len(sdv_synthetic_data["transfers"])
transfers_per_customer = synthetic_transfers_count / synthetic_customers_count if synthetic_customers_count > 0 else 0

print("\n🔍 Generation Quality Metrics:")
print(f"   - Transfers per customer: {transfers_per_customer:.1f}")
print(f"   - Scale factor achieved: {synthetic_customers_count / len(customers_train):.2f}x")

# Quick preview of generated synthetic data
print("\n📋 Sample of SDV synthetic customer data:")
print(sdv_synthetic_data["customers"].head())
print("\n📋 Sample of SDV synthetic transfer data:")
print(sdv_synthetic_data["transfers"].head())

In [ ]:
# Save SDV synthetic data for comparison
customers_output_file = "./data/sdv_customers.parquet"
transfers_output_file = "./data/sdv_transfers.parquet"
sdv_synthetic_data["customers"].to_parquet(customers_output_file, index=False)
sdv_synthetic_data["transfers"].to_parquet(transfers_output_file, index=False)
print(f"💾 SDV synthetic data saved to: {customers_output_file} and {transfers_output_file}")

## 5. MOSTLY AI Implementation

**About MOSTLY AI Synthetic Data SDK:**
- Open-source (Apache 2) synthetic data SDK with advanced AI capabilities
- Also cloud-based service with enterprise-grade security and compliance
- Supports complex multi-table scenarios with multiple foreign keys
- Uses deep learning and autoregressive-based models

**Getting Started:**
- **API Access**: Requires valid API credentials for cloud platform access
- **API Key Generation**: Get your free API key at: https://app.mostly.ai/settings/api-keys

**Key Advantages:**
- **Advanced AI Models**: Utilizes state-of-the-art generative AI including language models
- **Multi-Parent Support**: Can handle complex relationships (multiple foreign keys per table)
- **Mixed Data Types**: Excels at both tabular and text data synthesis
- **Enterprise Features**: Privacy guarantees, compliance reporting, and scalability

**Architecture:**
- **Tabular Models**: For structured data (demographics, financials)
- **Language Models**: For text fields (names, addresses, emails) using LLMs like Llama-3.2
- **Sequential Models**: For time-series and ordered data patterns

In [ ]:
from mostlyai.sdk import MostlyAI

print("🔧 Initializing MOSTLY AI Synthetic Data SDK...")

# Initialize MOSTLY AI Synthetic Data SDK
mostly = MostlyAI(local=True)


print("✅ MOSTLY AI Synthetic Data SDK initialized successfully")

### 5.1 MOSTLY AI Advanced Configuration

**Multi-Table Configuration Highlights:**

**Customers Table:**
- **Mixed Encoding Types**: Combines tabular (demographics) and language models (text fields)
- **PII Handling**: Specialized handling for names, SSN, addresses with privacy protection
- **Geographic Data**: Advanced lat/lon synthesis maintaining geographic coherence
- **Language Model**: Uses Llama-3.2-3B for realistic text generation

**Transfers Table:**
- **Dual Foreign Keys**: Both issuer_id and receiver_id properly modeled (unlike SDV limitation)
- **Sequential Data**: Timestamp modeling for realistic temporal patterns
- **Context Relationships**: issuer_id marked as context (provides additional context during generation)

**Advanced Features:**
- **Flexible Generation**: Disabled for consistent comparison
- **Model Reports**: Enabled for quality assessment
- **Training Time Limits**: 10 minutes per model for efficient comparison


In [ ]:
print("⚙️ Configuring advanced MOSTLY AI generator...")
print("Setting up sophisticated multi-table configuration with dual foreign keys...")

# Configure the generator for comprehensive multi-table setup
# This configuration showcases MOSTLY AI's advanced capabilities:
# - Mixed data types (tabular + language models)
# - Multiple foreign key relationships
# - Specialized encoding for different data types
config = {
    "name": "Customers & Transfers Generator",
    "tables": [
        {
            "name": "customers",
            "data": customers_train,
            "primary_key": "customer_id",
            "columns": [
                {"name": "customer_id", "model_encoding_type": "AUTO"},
                {"name": "ssn", "model_encoding_type": "LANGUAGE_TEXT"},
                {"name": "blood_group", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "username", "model_encoding_type": "LANGUAGE_TEXT"},
                {"name": "sex", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "mail", "model_encoding_type": "LANGUAGE_TEXT"},
                {"name": "address1", "model_encoding_type": "LANGUAGE_TEXT"},
                {"name": "address2", "model_encoding_type": "LANGUAGE_TEXT"},
                {"name": "city", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "postalCode", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "state", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "age", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "workclass", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "education", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "education-num", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "marital-status", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "occupation", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "relationship", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "race", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "capital-gain", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "capital-loss", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "hours-per-week", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "native-country", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "income", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "card_type", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "card_number", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "card_expire_date", "model_encoding_type": "TABULAR_CATEGORICAL"},
                {"name": "CVC", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "first_name", "model_encoding_type": "LANGUAGE_TEXT"},
                {"name": "last_name", "model_encoding_type": "LANGUAGE_TEXT"},
                {"name": "lat_lon", "model_encoding_type": "TABULAR_LAT_LONG"},
            ],
            "tabular_model_configuration": {
                "model": "MOSTLY_AI/Medium",
                "max_training_time": 30,
                "enable_flexible_generation": False,
                "value_protection": False,
                "enable_model_report": True,
            },
            "language_model_configuration": {
                "model": "MOSTLY_AI/LSTMFromScratch-3m",
                "max_training_time": 30,
                "enable_flexible_generation": False,
                "enable_model_report": True,
            },
        },
        {
            "name": "transfers",
            "data": transfers_train,
            "primary_key": "transfer_id",
            "foreign_keys": [
                {"column": "issuer_id", "referenced_table": "customers", "is_context": True},
                {"column": "receiver_id", "referenced_table": "customers", "is_context": False},
            ],
            "columns": [
                {"name": "transfer_id", "model_encoding_type": "AUTO"},
                {"name": "receiver_id", "model_encoding_type": "AUTO"},
                {"name": "issuer_id", "model_encoding_type": "AUTO"},
                {"name": "amount", "model_encoding_type": "TABULAR_NUMERIC_AUTO"},
                {"name": "timestamp", "model_encoding_type": "TABULAR_DATETIME"},
                {"name": "note", "model_encoding_type": "TABULAR_NUMERIC_DISCRETE"},
            ],
            "tabular_model_configuration": {
                "model": "MOSTLY_AI/Medium",
                "max_training_time": 30,
                "max_sequence_window": 10,
                "enable_flexible_generation": False,
                "value_protection": False,
                "enable_model_report": True,
            },
        },
    ],
}

### 5.2 MOSTLY AI Training Process

**Training Process:**
1. **Upload Data**: Send training data securely to MOSTLY AI cloud
2. **Model Configuration**: Apply the complex multi-table configuration
3. **AI Training**: Use advanced generative models including LLMs
4. **Quality Validation**: Automatic quality checks during training


In [ ]:
print("🚀 Starting MOSTLY AI training...")
print("📤 Uploading training data to secure MOSTLY AI cloud platform...")

start_time = time.time()

# Train the MOSTLY AI generator with our advanced configuration
# This will:
# 1. Upload training data securely to the cloud
# 2. Configure both tabular and language models
# 3. Train AI models for each table and their relationships
# 4. Wait for training completion with progress monitoring
g = mostly.train(config=config, start=True, wait=True)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print("✅ MOSTLY AI training completed successfully!")
print(f"⏱️ Total training time: {elapsed_minutes:.2f} minutes")
print("🧠 Advanced AI models trained for multi-table synthesis")

In [ ]:
print("🎲 Starting MOSTLY AI synthetic data generation...")
print("🌩️ Using cloud-based AI models for high-quality synthesis...")

start_time = time.time()

# Generate synthetic data using the trained MOSTLY AI generator
# Key advantages over SDV:
# - Handles dual foreign keys properly
# - Uses advanced language models for text fields
# - Maintains complex statistical relationships
print(f"📊 Generating {len(df_customers):,} synthetic customer records...")

sd = mostly.generate(g, size=len(df_customers))
mostlyai_synthetic_data = sd.data()

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

# Calculate generation statistics
total_records = len(mostlyai_synthetic_data["customers"]) + len(mostlyai_synthetic_data["transfers"])
generation_rate = total_records / (end_time - start_time)

print("✅ MOSTLY AI generation completed successfully!")
print(f"⏱️ Generation time: {elapsed_minutes:.2f} minutes")
print(f"🚀 Generation rate: {generation_rate:,.0f} records/second")
print(f"📊 Generated {len(mostlyai_synthetic_data['customers']):,} synthetic customers")
print(f"📊 Generated {len(mostlyai_synthetic_data['transfers']):,} synthetic transfers")

# Quality metrics
transfers_per_customer = len(mostlyai_synthetic_data["transfers"]) / len(mostlyai_synthetic_data["customers"])
print("\n🔍 Quality Metrics:")
print(f"   - Transfers per customer: {transfers_per_customer:.1f}")
print("   - Both foreign keys properly handled ✅")

# Quick preview of generated synthetic data
print("\n📋 Sample MOSTLY AI synthetic customer data:")
print(mostlyai_synthetic_data["customers"].head())
print("\n📋 Sample MOSTLY AI synthetic transfer data:")
print(mostlyai_synthetic_data["transfers"].head())

In [ ]:
# Save MOSTLY AI synthetic data for comparison
customers_output_file = "./data/mostlyai_customers.parquet"
transfers_output_file = "./data/mostlyai_transfers.parquet"
mostlyai_synthetic_data["customers"].to_parquet(customers_output_file, index=False)
mostlyai_synthetic_data["transfers"].to_parquet(transfers_output_file, index=False)
print(f"💾 MOSTLY AI synthetic data saved to: {customers_output_file} and {transfers_output_file}")

## 6. Synthetic Data Quality Assessment

After generating synthetic data using both SDV and MOSTLY AI, it's crucial to comprehensively evaluate the quality, privacy, and integrity of the generated datasets. This section provides a multi-faceted quality assessment framework that ensures our synthetic data meets production standards.

### **Quality Assessment Framework:**

Our evaluation methodology consists of two complementary approaches:

1. **Statistical Quality Assessment (Section 6.1)**: Using the MOSTLY AI QA library to evaluate statistical fidelity, privacy metrics, and overall data quality
2. **Foreign Key Integrity Verification (Section 6.2)**: Custom verification to ensure referential integrity and relationship preservation in multi-table synthetic data

### **Key Quality Dimensions Evaluated:**

- **📊 Statistical Accuracy**: How well synthetic data preserves statistical properties of the original data
- **🔒 Privacy Protection**: Measurement of privacy risks and distance to closest record (DCR)
- **🔗 Referential Integrity**: Verification that foreign key relationships are maintained correctly
- **📈 Coverage Analysis**: Assessment of how comprehensively synthetic data represents the original data space
- **⚖️ Utility vs Privacy Balance**: Evaluation of the trade-off between data utility and privacy protection

## 6.1 Statistical Quality Assessment with MOSTLY AI QA Library

The MOSTLY AI QA library provides enterprise-grade quality assessment capabilities that evaluate synthetic data across multiple dimensions. This assessment generates comprehensive HTML reports and quantitative metrics that help understand:

- **Accuracy Scores**: Overall statistical fidelity of synthetic data
- **Distance to Closest Record (DCR)**: Privacy risk measurement 
- **Univariate & Bivariate Distributions**: Preservation of individual column and column-pair statistics
- **Correlation Analysis**: Maintenance of relationships between variables
- **Similarity Metrics**: Overall resemblance to training data while avoiding overfitting

These assessments are performed for both customer and transfer datasets, comparing synthetic data against both training and holdout datasets to ensure robust evaluation.

In [ ]:
# Import and initialize the quality assessment framework
from mostlyai import qa

# Initialize logging to see detailed evaluation progress
qa.init_logging()
print("🔍 Quality assessment framework initialized")

In [15]:
# Load the split files from the disk
customers_train = pd.read_parquet("./data/customers_train.parquet")
customers_test = pd.read_parquet("./data/customers_test.parquet")
transfers_train = pd.read_parquet("./data/transfers_train.parquet")
transfers_test = pd.read_parquet("./data/transfers_test.parquet")

In [ ]:
print("📊 Evaluating SDV Customers synthetic data quality...")

# Load the SDV synthetic dataset
sdv_customers = pd.read_parquet("./data/sdv_customers.parquet")

# Run comprehensive quality assessment
# This compares synthetic data against training and holdout sets
report_path, metrics = qa.report(
    syn_tgt_data=sdv_customers,  # SDV synthetic data
    trn_tgt_data=customers_train,  # Original training data
    hol_tgt_data=customers_test,  # Holdout data for validation
    max_sample_size_embeddings=10_000,  # Limit sample size for efficiency
    report_path="sdv_customers_qa_report.html",  # HTML report output
)

print(f"📋 SDV Customers Quality Report saved to: {report_path}")
print("\n📈 SDV Customers Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Extract key metrics for comparison
sdv_customers_accuracy = metrics.accuracy.overall
sdv_customers_dcr_share = metrics.distances.dcr_share
print("\n🎯 SDV Customers Summary:")
print(f"   Overall Accuracy: {sdv_customers_accuracy:.3f}")
print(f"   DCR Share: {sdv_customers_dcr_share:.3f}")

In [ ]:
print("📊 Evaluating SDV Tranfers synthetic data quality...")

# Load the SDV synthetic dataset
sdv_transfers = pd.read_parquet("./data/sdv_transfers.parquet")

# Define ID columns to exclude from QA analysis
id_columns_to_exclude = ["customer_id", "transfer_id", "receiver_id"]


# Create copies and remove ID columns for QA
def remove_id_columns(df, columns_to_remove):
    """Remove specified columns if they exist in the dataframe"""
    return df.drop(columns=[col for col in columns_to_remove if col in df.columns])


# Prepare transfers data (remove ID columns)
sdv_transfers = remove_id_columns(sdv_transfers, id_columns_to_exclude)
transfers_train_qa = remove_id_columns(transfers_train, id_columns_to_exclude)
transfers_test_qa = remove_id_columns(transfers_test, id_columns_to_exclude)

# Run comprehensive quality assessment
# This compares synthetic data against training and holdout sets
report_path, metrics = qa.report(
    syn_tgt_data=sdv_transfers,
    trn_tgt_data=transfers_train_qa,
    hol_tgt_data=transfers_test_qa,
    syn_ctx_data=sdv_customers,
    trn_ctx_data=customers_train,
    hol_ctx_data=customers_test,
    ctx_primary_key="customer_id",
    tgt_context_key="issuer_id",
    max_sample_size_embeddings=10_000,  # Limit sample size for efficiency
    report_path="sdv_transfers_qa_report.html",  # HTML report output
)

print(f"📋 SDV Transfers Quality Report saved to: {report_path}")
print("\n📈 SDV Transfers Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Extract key metrics for comparison
sdv_transfers_accuracy = metrics.accuracy.overall
sdv_transfers_dcr_share = metrics.distances.dcr_share
print("\n🎯 SDV Transfers Summary:")
print(f"   Overall Accuracy: {sdv_transfers_accuracy:.3f}")
print(f"   DCR Share: {sdv_transfers_dcr_share:.3f}")

In [ ]:
print("📊 Evaluating MOSTLY AI Customers synthetic data quality...")

# Load the SDV synthetic dataset
mostlyai_customers = pd.read_parquet("./data/mostlyai_customers.parquet")

# Run comprehensive quality assessment
# This compares synthetic data against training and holdout sets
report_path, metrics = qa.report(
    syn_tgt_data=mostlyai_customers,  # SDV synthetic data
    trn_tgt_data=customers_train,  # Original training data
    hol_tgt_data=customers_test,  # Holdout data for validation
    max_sample_size_embeddings=10_000,  # Limit sample size for efficiency
    report_path="mostlyai_customers_qa_report.html",  # HTML report output
)

print(f"📋 MOSTLY AI Customers Quality Report saved to: {report_path}")
print("\n📈 MOSTLY AI Customers Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Extract key metrics for comparison
mostlyai_customers_accuracy = metrics.accuracy.overall
mostlyai_customers_dcr_share = metrics.distances.dcr_share
print("\n🎯 MOSTLY AI Customers Summary:")
print(f"   Overall Accuracy: {mostlyai_customers_accuracy:.3f}")
print(f"   DCR Share: {mostlyai_customers_dcr_share:.3f}")

In [ ]:
print("📊 Evaluating MOSTLY AI Transfers synthetic data quality...")

# Load the MOSTLY AI synthetic dataset
mostlyai_transfers = pd.read_parquet("./data/mostlyai_transfers.parquet")

# Define ID columns to exclude from QA analysis
id_columns_to_exclude = ["customer_id", "transfer_id", "receiver_id"]


# Create copies and remove ID columns for QA
def remove_id_columns(df, columns_to_remove):
    """Remove specified columns if they exist in the dataframe"""
    return df.drop(columns=[col for col in columns_to_remove if col in df.columns])


# Prepare transfers data (remove ID columns)
mostlyai_transfers = remove_id_columns(mostlyai_transfers, id_columns_to_exclude)
transfers_train_qa = remove_id_columns(transfers_train, id_columns_to_exclude)
transfers_test_qa = remove_id_columns(transfers_test, id_columns_to_exclude)

# Run comprehensive quality assessment
# This compares synthetic data against training and holdout sets
report_path, metrics = qa.report(
    syn_tgt_data=mostlyai_transfers,
    trn_tgt_data=transfers_train_qa,
    hol_tgt_data=transfers_test_qa,
    syn_ctx_data=mostlyai_customers,
    trn_ctx_data=customers_train,
    hol_ctx_data=customers_test,
    ctx_primary_key="customer_id",
    tgt_context_key="issuer_id",
    max_sample_size_embeddings=10_000,  # Limit sample size for efficiency
    report_path="mostlyai_transfers_qa_report.html",  # HTML report output
)

print(f"📋 MOSTLY AI Transfers Quality Report saved to: {report_path}")
print("\n📈 MOSTLY AI Transfers Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Extract key metrics for comparison
mostlyai_transfers_accuracy = metrics.accuracy.overall
mostlyai_transfers_dcr_share = metrics.distances.dcr_share
print("\n🎯 MOSTLY AI Transfers Summary:")
print(f"   Overall Accuracy: {mostlyai_transfers_accuracy:.3f}")
print(f"   DCR Share: {mostlyai_transfers_dcr_share:.3f}")

In [ ]:
# Add a final comparison section
print("\n" + "=" * 60)
print("🏆 FINAL COMPARISON")
print("=" * 60)
print(f"SDV Customers       - Accuracy: {sdv_customers_accuracy:.3f}, DCR Share: {sdv_customers_dcr_share:.3f}")
print(f"SDV Transfers       - Accuracy: {sdv_transfers_accuracy:.3f}, DCR Share: {sdv_transfers_dcr_share:.3f}")
print(
    f"MOSTLY AI Customers - Accuracy: {mostlyai_customers_accuracy:.3f}, DCR Share: {mostlyai_customers_dcr_share:.3f}"
)
print(
    f"MOSTLY AI Transfers - Accuracy: {mostlyai_transfers_accuracy:.3f}, DCR Share: {mostlyai_transfers_dcr_share:.3f}"
)
print("\n🔍 METRIC INTERPRETATION:")
print("• Higher accuracy = better statistical fidelity")
print("• DCR Share ~0.5 = optimal privacy-utility balance")
print("\n📊 ANALYSIS:")
print("• MOSTLY AI shows significantly higher accuracy")
print("• Both frameworks achieve good DCR Share balance (~0.5)")
print("• MOSTLY AI demonstrates superior statistical fidelity while maintaining privacy")
print("\n⚠️  RECOMMENDATION:")
print("• Review detailed HTML reports for comprehensive privacy assessment")
print("• Consider discriminator AUC and similarity metrics for additional insights")
print("• Evaluate based on your specific privacy-utility requirements")

## 6.2 Foreign Key Integrity Verification

One critical aspect of multi-table synthetic data quality is ensuring that foreign key relationships are maintained properly. This section verifies that:

1. **Referential Integrity**: All foreign keys in synthetic transfers reference valid customer IDs
2. **Coverage**: All synthetic customers are properly referenced in the transfers table
3. **Relationship Patterns**: The distribution of transfers per customer matches expected patterns

This verification is essential for downstream applications that rely on proper table relationships.

In [ ]:
print("🔍 Performing Foreign Key Integrity Verification...")
print("=" * 60)


def verify_foreign_key_integrity(customers_df, transfers_df, provider_name):
    """
    Comprehensive foreign key integrity verification for synthetic data

    Args:
        customers_df: Synthetic customers dataframe
        transfers_df: Synthetic transfers dataframe
        provider_name: Name of the synthetic data provider (e.g., 'SDV', 'MOSTLY AI')

    Returns:
        Dictionary with integrity metrics
    """
    print(f"\n🏷️  {provider_name} Foreign Key Verification:")
    print("-" * 40)

    # Make sure all columns are the same type
    customers_df["customer_id"] = customers_df["customer_id"].astype(str)
    transfers_df["issuer_id"] = transfers_df["issuer_id"].astype(str)
    transfers_df["receiver_id"] = transfers_df["receiver_id"].astype(str)

    # Get customer IDs from both datasets
    customer_ids = set(customers_df["customer_id"].unique())
    issuer_ids = set(transfers_df["issuer_id"].unique())
    receiver_ids = set(transfers_df["receiver_id"].unique())

    # All foreign key IDs referenced in transfers
    all_transfer_fk_ids = issuer_ids.union(receiver_ids)

    # Verification 1: Referential Integrity
    invalid_issuer_ids = issuer_ids - customer_ids
    invalid_receiver_ids = receiver_ids - customer_ids
    invalid_fk_ids = all_transfer_fk_ids - customer_ids

    referential_integrity = len(invalid_fk_ids) == 0

    # Verification 2: Coverage Analysis
    referenced_customers = len(all_transfer_fk_ids)
    total_customers = len(customer_ids)
    coverage_percentage = (referenced_customers / total_customers) * 100

    # Verification 3: Transfer Distribution Analysis
    issuer_transfer_counts = transfers_df["issuer_id"].value_counts()
    receiver_transfer_counts = transfers_df["receiver_id"].value_counts()

    # Calculate statistics
    avg_transfers_per_issuer = issuer_transfer_counts.mean()
    avg_transfers_per_receiver = receiver_transfer_counts.mean()
    max_transfers_per_issuer = issuer_transfer_counts.max()
    min_transfers_per_issuer = issuer_transfer_counts.min()

    # Results Summary
    metrics = {
        "provider": provider_name,
        "total_customers": total_customers,
        "total_transfers": len(transfers_df),
        "unique_issuers": len(issuer_ids),
        "unique_receivers": len(receiver_ids),
        "referenced_customers": referenced_customers,
        "coverage_percentage": coverage_percentage,
        "invalid_issuer_ids": len(invalid_issuer_ids),
        "invalid_receiver_ids": len(invalid_receiver_ids),
        "total_invalid_fks": len(invalid_fk_ids),
        "referential_integrity": referential_integrity,
        "avg_transfers_per_issuer": avg_transfers_per_issuer,
        "avg_transfers_per_receiver": avg_transfers_per_receiver,
        "max_transfers_per_issuer": max_transfers_per_issuer,
        "min_transfers_per_issuer": min_transfers_per_issuer,
    }

    # Print detailed results
    print("📊 Dataset Summary:")
    print(f"   • Total Customers: {total_customers:,}")
    print(f"   • Total Transfers: {len(transfers_df):,}")
    print(f"   • Unique Issuers: {len(issuer_ids):,}")
    print(f"   • Unique Receivers: {len(receiver_ids):,}")

    print("\n🔗 Referential Integrity:")
    if referential_integrity:
        print("   ✅ PASSED - All foreign keys reference valid customers")
    else:
        print(f"   ❌ FAILED - {len(invalid_fk_ids):,} invalid foreign key references found")
        if invalid_issuer_ids:
            print(f"   • Invalid issuer_ids: {len(invalid_issuer_ids):,}")
        if invalid_receiver_ids:
            print(f"   • Invalid receiver_ids: {len(invalid_receiver_ids):,}")

    print("\n📈 Coverage Analysis:")
    print(f"   • Customers Referenced: {referenced_customers:,} ({coverage_percentage:.1f}%)")
    print(f"   • Customers Not Referenced: {total_customers - referenced_customers:,}")

    print("\n📊 Transfer Distribution:")
    print(f"   • Avg Transfers per Issuer: {avg_transfers_per_issuer:.1f}")
    print(f"   • Avg Transfers per Receiver: {avg_transfers_per_receiver:.1f}")
    print(f"   • Max Transfers per Issuer: {max_transfers_per_issuer:,}")
    print(f"   • Min Transfers per Issuer: {min_transfers_per_issuer:,}")

    return metrics


# Load synthetic datasets for verification
print("📂 Loading synthetic datasets...")

# SDV Synthetic Data
sdv_customers = pd.read_parquet("./data/sdv_customers.parquet")
sdv_transfers = pd.read_parquet("./data/sdv_transfers.parquet")

# MOSTLY AI Synthetic Data
mostlyai_customers = pd.read_parquet("./data/mostlyai_customers.parquet")
mostlyai_transfers = pd.read_parquet("./data/mostlyai_transfers.parquet")

print("✅ Synthetic datasets loaded successfully")

In [ ]:
# Perform foreign key verification for both providers
print("🚀 Running Foreign Key Integrity Verification for Both Providers...")

# Verify SDV synthetic data integrity
sdv_metrics = verify_foreign_key_integrity(customers_df=sdv_customers, transfers_df=sdv_transfers, provider_name="SDV")

# Verify MOSTLY AI synthetic data integrity
mostlyai_metrics = verify_foreign_key_integrity(
    customers_df=mostlyai_customers, transfers_df=mostlyai_transfers, provider_name="MOSTLY AI"
)

In [ ]:
# Comparative Analysis and Summary
print("\n" + "=" * 80)
print("🏆 FOREIGN KEY INTEGRITY COMPARISON SUMMARY")
print("=" * 80)

# Create comparison dataframe
comparison_data = []
for metrics in [sdv_metrics, mostlyai_metrics]:
    comparison_data.append(
        {
            "Provider": metrics["provider"],
            "Total Customers": f"{metrics['total_customers']:,}",
            "Total Transfers": f"{metrics['total_transfers']:,}",
            "Referential Integrity": "✅ PASS" if metrics["referential_integrity"] else "❌ FAIL",
            "Invalid FKs": f"{metrics['total_invalid_fks']:,}",
            "Coverage %": f"{metrics['coverage_percentage']:.1f}%",
            "Avg Transfers/Issuer": f"{metrics['avg_transfers_per_issuer']:.1f}",
            "Max Transfers/Issuer": f"{metrics['max_transfers_per_issuer']:,}",
        }
    )

comparison_df = pd.DataFrame(comparison_data)
print("\n📊 Side-by-Side Comparison:")
print(comparison_df.to_string(index=False))

# Key Insights
print("\n🔍 Key Insights:")

# Referential Integrity Comparison
if sdv_metrics["referential_integrity"] and mostlyai_metrics["referential_integrity"]:
    print("   ✅ Both providers maintain perfect referential integrity")
elif sdv_metrics["referential_integrity"]:
    print("   ⚠️  SDV maintains better referential integrity than MOSTLY AI")
elif mostlyai_metrics["referential_integrity"]:
    print("   ⚠️  MOSTLY AI maintains better referential integrity than SDV")
else:
    print("   ❌ Both providers have referential integrity issues")

# Coverage Comparison
coverage_diff = mostlyai_metrics["coverage_percentage"] - sdv_metrics["coverage_percentage"]
if abs(coverage_diff) < 5:
    print("   📊 Similar customer coverage between both providers")
elif coverage_diff > 0:
    print(f"   📈 MOSTLY AI has {coverage_diff:.1f}% higher customer coverage than SDV")
else:
    print(f"   📈 SDV has {abs(coverage_diff):.1f}% higher customer coverage than MOSTLY AI")

# Transfer Distribution Comparison
sdv_avg = sdv_metrics["avg_transfers_per_issuer"]
mostlyai_avg = mostlyai_metrics["avg_transfers_per_issuer"]
if abs(sdv_avg - mostlyai_avg) < 5:
    print("   🔄 Similar transfer distribution patterns")
elif mostlyai_avg > sdv_avg:
    print(f"   🔄 MOSTLY AI generates {mostlyai_avg - sdv_avg:.1f} more transfers per issuer on average")
else:
    print(f"   🔄 SDV generates {sdv_avg - mostlyai_avg:.1f} more transfers per issuer on average")

print("\n✨ Foreign Key Integrity Verification Complete!")
print("=" * 80)

## 7. Comprehensive Comparison: SDV vs MOSTLY AI

### 7.1 Technical Capabilities Comparison

| Feature | SDV (Business Source) | MOSTLY AI (Open-Source / Cloud based) |
|---------|-------------------|----------------------|
| **Multi-table Support** | ✅ Yes (HMASynthesizer) | ✅ Yes (Advanced) |
| **Multiple Foreign Keys** | ❌ One parent per child only | ✅ Full support |
| **Text Generation** | ⚠️ Basic categorical | ✅ Advanced LLM-based |
| **Mixed Data Types** | ✅ Yes | ✅ Yes (Superior) |
| **Training Time** | ~4 minutes | ~30 minutes |
| **Privacy Controls** | ✅ Basic | ✅ Enterprise-grade |
| **Deployment** | 🏠 Local/Self-hosted | 🏠 Local/Self-hosted or ☁️ Cloud-based |
| **Cost** | 🆓 Free | 🆓 Free |

### 7.2 Data Quality Observations

**SDV Strengths:**
- Fast local training and generation
- Good statistical preservation for numerical data
- Strong community and documentation
- Full control over data and models

**SDV Limitations:**
- Cannot model dual foreign keys properly (issuer_id treated as regular column)
- Basic text synthesis capabilities
- Limited privacy protection features

**MOSTLY AI Strengths:**
- Fast local training and generation
- Advanced AI models including language models
- Proper handling of complex relationships
- Superior text field synthesis (names, addresses, emails)
- Enterprise privacy and compliance features

## 8. Alternative: MOSTLY AI Mock Data Generation

As an alternative to training complex models, MOSTLY AI also provides a mock data generation capability that can quickly create synthetic data based on prompts and schemas. This is useful for:

- **Rapid Prototyping**: Quick synthetic data without model training
- **Development Testing**: Generate test data on-demand
- **Schema Validation**: Test data pipelines with realistic data structures

The following section demonstrates this alternative approach using detailed table schemas and business logic prompts.


In [ ]:
# Alternative approach: Mock data generation without model training
# This method uses prompt-based generation for rapid prototyping
print("🎭 Setting up MOSTLY AI mock data generation...")

from mostlyai import mock

# Define comprehensive table schemas with detailed prompts
# This approach relies on AI understanding of the prompts rather than learned patterns
print("📝 Defining detailed table schemas with business logic...")

tables = {
    "customers": {
        "prompt": "Customers of a financial service",
        "columns": {
            "customer_id": {"prompt": "the unique id of the customer", "dtype": "integer"},
            "ssn": {"prompt": "US social security number, e.g. 709-42-8435, 523-40-2158", "dtype": "string"},
            "blood_group": {"dtype": "category", "values": ["B-", "AB-", "O+", "AB+", "B+", "A-", "A+", "O-"]},
            "username": {"prompt": "username of the customer", "dtype": "string"},
            "sex": {"dtype": "category", "values": ["M", "F"]},
            "mail": {"prompt": "email address", "dtype": "string"},
            "address1": {"prompt": "primary address", "dtype": "string"},
            "address2": {"prompt": "secondary address", "dtype": "string"},
            "city": {"prompt": "city name, e.g. Fremont, Glendale, Panama City, Lowell, Manchester", "dtype": "string"},
            "postalCode": {"prompt": "postal code", "dtype": "integer"},
            "state": {
                "dtype": "category",
                "values": [
                    "CA",
                    "AZ",
                    "FL",
                    "MA",
                    "CT",
                    "TN",
                    "AL",
                    "AK",
                    "VT",
                    "KY",
                    "AR",
                    "CO",
                    "GA",
                    "DC",
                    "OK",
                    "MD",
                ],
            },
            "age": {"prompt": "age in years", "dtype": "integer"},
            "workclass": {
                "dtype": "category",
                "values": [
                    " Self-emp-not-inc",
                    " Private",
                    " Local-gov",
                    " State-gov",
                    " ?",
                    " Self-emp-inc",
                    " Federal-gov",
                    " Without-pay",
                ],
            },
            "education": {
                "dtype": "category",
                "values": [
                    " Bachelors",
                    " HS-grad",
                    " Some-college",
                    " 11th",
                    " 10th",
                    " 12th",
                    " Masters",
                    " Assoc-voc",
                    " 7th-8th",
                    " Doctorate",
                    " Prof-school",
                    " Assoc-acdm",
                    " 9th",
                    " 5th-6th",
                    " 1st-4th",
                    " Preschool",
                ],
            },
            "education-num": {"prompt": "education number", "dtype": "integer"},
            "marital-status": {
                "dtype": "category",
                "values": [
                    " Married-civ-spouse",
                    " Never-married",
                    " Married-spouse-absent",
                    " Widowed",
                    " Divorced",
                    " Separated",
                    " Married-AF-spouse",
                ],
            },
            "occupation": {
                "dtype": "category",
                "values": [
                    " Sales",
                    " Other-service",
                    " Craft-repair",
                    " Machine-op-inspct",
                    " Adm-clerical",
                    " Transport-moving",
                    " ?",
                    " Prof-specialty",
                    " Handlers-cleaners",
                    " Farming-fishing",
                    " Exec-managerial",
                    " Tech-support",
                    " Armed-Forces",
                    " Protective-serv",
                    " Priv-house-serv",
                ],
            },
            "relationship": {
                "dtype": "category",
                "values": [" Husband", " Own-child", " Unmarried", " Not-in-family", " Wife", " Other-relative"],
            },
            "race": {
                "dtype": "category",
                "values": [" Asian-Pac-Islander", " Black", " White", " Amer-Indian-Eskimo", " Other"],
            },
            "capital-gain": {"prompt": "capital gain", "dtype": "integer"},
            "capital-loss": {"prompt": "capital loss", "dtype": "integer"},
            "hours-per-week": {"prompt": "hours worked per week", "dtype": "integer"},
            "native-country": {
                "prompt": "country name, e.g. United-States, China, Japan, Germany, India",
                "dtype": "string",
            },
            "income": {"dtype": "category", "values": [" >50K", " <=50K"]},
            "card_type": {
                "dtype": "category",
                "values": [
                    "JCB 16 digit",
                    "Discover",
                    "VISA 16 digit",
                    "Mastercard",
                    "VISA 13 digit",
                    "VISA 19 digit",
                    "Maestro",
                    "American Express",
                    "Diners Club / Carte Blanche",
                    "JCB 15 digit",
                ],
            },
            "card_number": {
                "prompt": "credit card number, e.g. 3568039962967044, 3527524392405483",
                "dtype": "integer",
            },
            "card_expire_date": {"prompt": "credit card expiration date, e.g. 11/22, 10/30, 01/30", "dtype": "string"},
            "CVC": {"prompt": "credit card CVC", "dtype": "integer"},
            "first_name": {"prompt": "first name", "dtype": "string"},
            "last_name": {"prompt": "last name", "dtype": "string"},
            "lat_lon": {
                "prompt": "latitude and longitude as a comma-separated string, e.g. 37.5666441,-122.0444344, 33.5125581,-112.1828849, 33.525117,-112.215039",
                "dtype": "string",
            },
        },
        "primary_key": "customer_id",
    },
    "transfers": {
        "prompt": "Money transfers between customers",
        "columns": {
            "transfer_id": {"prompt": "the unique id of the transfer", "dtype": "integer"},
            "issuer_id": {"prompt": "the customer id of the sender", "dtype": "integer"},
            "receiver_id": {"prompt": "the customer id of the receiver", "dtype": "integer"},
            "amount": {"prompt": "amount transferred in USD", "dtype": "integer"},
            "timestamp": {
                "prompt": "transfer timestamp, e.g. 2019-01-01 00:00:34, 2019-01-01 00:06:45, 2019-01-01 00:19:58",
                "dtype": "datetime",
            },
            "note": {"prompt": "transfer note code", "dtype": "integer"},
        },
        "primary_key": "transfer_id",
        "foreign_keys": [
            {
                "column": "issuer_id",
                "referenced_table": "customers",
                "prompt": "each customer issues between 1 and 5 transfers",
            },
            {"column": "receiver_id", "referenced_table": "customers"},
        ],
    },
}

In [ ]:
print("🚀 Generating mock data using AI prompts...")
print("🤖 Using advanced language model for schema-based generation...")

# Generate synthetic data using prompt-based AI generation
# This approach is faster than model training but may be less statistically accurate
data = mock.sample(
    tables=tables,
    sample_size=100,  # Number of customers to generate; transfers will be generated per FK prompt
    model="openrouter/meta-llama/llama-4-scout:nitro",
    api_key="YOUR_API_KEY_HERE",
)

print("✅ Mock data generation completed!")
print(f"📊 Generated {len(data['customers'])} customers and {len(data['transfers'])} transfers")

# Access the generated DataFrames for analysis
df_customers = data["customers"]
df_transfers = data["transfers"]

print("\n📋 Sample mock customer data:")
print(df_customers.head())
print("\n📋 Sample mock transfer data:")
print(df_transfers.head())

print("\n🔍 Mock Data Summary:")
print("   - Method: Prompt-based AI generation")
print("   - Training time: None (instant generation)")
print("   - Accuracy: Schema-compliant, may lack statistical patterns")
print("   - Use case: Rapid prototyping and testing")

## 9. Conclusion and Next Steps

This comprehensive comparison demonstrates three distinct approaches to synthetic data generation:

### **Summary of Methods Evaluated:**

1. **SDV (Statistical Approach)**: Fast, local, and statistically sound for simpler scenarios
2. **MOSTLY AI (AI-Powered)**: Advanced local and cloud-based solution with superior handling of complex relationships
3. **Mock Generation (Prompt-Based)**: Rapid prototyping solution for development and testing

### **Key Takeaways:**

- **Multi-table synthetic data generation** requires careful consideration of foreign key relationships
- **Data splitting strategies** must maintain referential integrity across related tables  
- **Choice of tool** depends on complexity requirements, budget, and deployment constraints
- **Each approach** has distinct strengths suitable for different use cases

### **Recommended Steps:**

1. **Quality Assessment**: Implement comprehensive evaluation metrics for synthetic data quality
2. **Privacy Analysis**: Conduct privacy audits to ensure synthetic data doesn't leak sensitive information
3. **Business Validation**: Verify that synthetic data maintains business logic and domain-specific constraints
4. **Performance Benchmarking**: Compare synthetic data performance in downstream ML models or analyses

---

**📚 Resources:**
- [SDV Documentation](https://docs.sdv.dev/)
- [MOSTLY AI Platform](https://mostly.ai/)
- [MOSTLY AI Synthetic Data SDK](https://github.com/mostly-ai/mostlyai)
- [MOSTLY AI Synthetic Mock Data](https://github.com/mostly-ai/mostlyai-mock)
